# Analyzing CIA Factbook Data Using SQL

### Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/)(2015), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

- population - The global population
- population_growth - the annual population growth rate, as a percentage
- area - the total land and water area

The SQLite `factbook.db` database can be downloaded [here.](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)

We'll use the following code to connect our jupyter notebook to our database file:

In [1]:
%reload_ext sql


In [2]:
%sql sqlite:///factbook.db

To run SQL queries in this project, we add **%%sql** on its own line to start of every query. 

### Data Overview

Let's start by getting information on the tables in the database.

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
  FROM sqlite_sequence;

 * sqlite:///factbook.db
Done.


name,seq
facts,261


As we can see above, apart from an internal automatically created table `sqlite_sequence`, we have a table called `facts` with 261 rows, with which we're going to work in this project.

Let's have a look at the first five rows:

In [5]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


As said in the intro, the `facts` table contains demographic and geographic data about all the countries. Since the table contains 261 rows, and since in 2015(the year when this factbook was released) there were 198 countries. Some of the rows must represent other territories or administrative units rather than countries.

The coulmn names are mostly self-explantory, here are descriptions for those less evident:

- area_land - the country's land area in square kilometers.
- area_water - the country's water area in square kilometers.
- population_growth - the country's population growth as a percentage.
- birth_rate - the number of births per year per 1,000 people.
- death_rate - the number of death per year per 1,000 people.
- migration_rate - the difference between the number of people entering(immigrants) and leaving(emigrants) a country during the year, per 1,000 people.

## Exploring Population Statistics

Let's start by calculating some summary statistics for `population`and look for any outlier countries.

In [6]:
%%sql
SELECT min(population),
       max(population),
       min(population_growth),
       max(population_growth),
       round(avg(population))
  FROM facts

 * sqlite:///factbook.db
Done.


min(population),max(population),min(population_growth),max(population_growth),round(avg(population))
0,7256490011,0.0,4.02,62094928.0


We see a few interesting things in the summary statistics :

- There's a country with a population equal to 0.
- There's a country with a population of 7.2 billion people, that seems to be the [world population in 2015](https://population.un.org/wpp/Publications/Files/WPP2015_DataBooklet.pdf)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [7]:
%%sql
SELECT *
  FROM facts
WHERE population in (SELECT min(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


This number for Antartica looks reasonable, since, according to the [CIA Factbook page for Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society), no indigenous inhabitants, but there are both permanent and summer-only staffed research stations


In [8]:
%%sql
SELECT *
  FROM facts
WHERE population in (SELECT max(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Hence, we can confirm that the maximum value of `population` just corresponds to the world population in 2015, represented by its own row in the table.

Now that we know this, we should recalculate the summary statistics we calculated earlier- this time excluding the row for the whole world

In [9]:
%%sql
SELECT min(population),   
       max(population), 
       min(population_growth), 
       max(population_growth),
       round(avg(population), 0)
    
FROM facts
WHERE name <> 'World'
  AND NAME <> 'Antartctica';

 * sqlite:///factbook.db
Done.


min(population),max(population),min(population_growth),max(population_growth),"round(avg(population), 0)"
0,1367485388,0.0,4.02,32242667.0


Now the minimum and maximum values look much more plausible. Also, the average population without the outliers decreased to almost half of the previous number.

Let's find out what countries have the largest and lowest populations. This time we shall take the look at top 5 and bottom 5 countries.

In [10]:
%%sql
SELECT name, population AS top5_population
  FROM facts
WHERE name <> 'World'
  AND name <> 'Antarctica'
ORDER BY population DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,top5_population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


One more issue appears here: the **European Union** is not a country, we have to exclude this as well

In [11]:
%%sql
SELECT name, population AS top5_population
  FROM facts
WHERE name <> 'World'
  AND name <> 'Antarctica'
  AND name <> 'European Union'
ORDER BY population DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,top5_population
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812


We see that most populated country is **China**(Which is a commonly known fact), followed by **India**, then with a huge gap followed by **United States**, **Indonesia** and **Brazil**.

In [12]:
%%sql
SELECT name, population AS bottom5_population
  FROM facts
WHERE name <> 'Antarctica'
  AND population NOT NULL
ORDER BY population
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,bottom5_population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337


Looks coherent for the bottom 5 countries. The [Pitcairn Islands](https://en.wikipedia.org/wiki/Pitcairn_Islands) is a group of four volcanic islands in the southern Pacific Ocean that form the sole British Overseas Territory in the Pacific Ocean. Pitcairn is the least populous national jurisdiction in the world.
**Niue** and **Tokelau** are alos very isolated islands in the southern Pacific Ocean, while **Cocos Islands** - in the Indian Ocean. Among these, not surprisingly, there is a microstate **Vatican**.

In our database, there are some territories with null valued `population`.


In [13]:
%%sql
SELECT name, population
 FROM facts
WHERE population IS NULL;

 * sqlite:///factbook.db
Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None


All these territories represent uninhabited islands, or oceans.

## Exploring Population Density Statistics 



In this section, we're going to find out which countries are the most densely populated and which ones the least.

First, let's calculate the average population density in the world, which practically means dividing `population` by `area`. Here and later, we have to:

- use `area_land` instead of `area`, for being more precise
- exclude null values for both `population` and `area_land`
- exclude `area_land`=0 to avoid having 0 in the denominator
- exclude the outliers World, European Union, Antartica.

Let's have a look once at the outliers again.

In [14]:
%%sql
SELECT name, population, area_land
  FROM facts
WHERE name = 'World'
   OR name = 'European Union'
   OR name = 'Antarctica';

 * sqlite:///factbook.db
Done.


name,population,area_land
European Union,513949445,None
Antarctica,0,280000
World,7256490011,None


Since according to the result above, we have to keep in mind only the row for `Antarctica`, as the other two outliers have null values for `area_land`.

Now we're ready to calculate the average values for the `population`, `area_land` and `population density`

In [15]:
%%sql
SELECT ROUND(AVG(population),0) AS avg_population,
       ROUND(AVG(area_land),0) AS avg_area_land,
       ROUND(AVG(population)/AVG(area_land), 0) AS avg_population_density
  FROM facts
 WHERE population NOT NULL
   AND area_land NOT NULL
   AND area_land <> 0
   AND name <> 'Antatctica';
    


 * sqlite:///factbook.db
Done.


avg_population,avg_area_land,avg_population_density
30510198.0,551845.0,55.0


Next, we're going to count the countries with the `population density` above average. We can assume that there will be roughly half of such countries in our database. So let's begin counting this figure.

In [16]:
%%sql
SELECT count(name) AS count_avg_population_density
  FROM facts
 WHERE population NOT NULL
   AND area_land NOT NULL
    AND area_land <> 0
     AND name <> 'Antarctica'
        AND population/area_land > (
            SELECT AVG(population)/AVG(area_land) 
              FROM facts
               WHERE population NOT NULL
                AND area_land NOT NULL
                AND area_land <> 0
                AND name <> 'Antarctica' 
                    );
        
        

 * sqlite:///factbook.db
Done.


count_avg_population_density
149


Hence, our assumption was right to not to visualize the data, so now let's only look at the *first 20*. 

In [17]:
%%sql
SELECT name, population, area_land, population/area_land AS population_density  
  FROM facts
 WHERE population NOT NULL
   AND area_land NOT NULL
    AND area_land <> 0
     AND name <> 'Antarctica'
        AND population/area_land > (
            SELECT AVG(population)/AVG(area_land) 
              FROM facts
               WHERE population NOT NULL
                AND area_land NOT NULL
                AND area_land <> 0
                AND name <> 'Antarctica')
 ORDER BY population_density DESC
 LIMIT 20;

 * sqlite:///factbook.db
Done.


name,population,area_land,population_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,687,8259
Hong Kong,7141106,1073,6655
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


According to the result, the `highest population density` is observed in **Macau**, followed with a huge gap by **Monaco**, which is then followed with another huge gap by **Singapore** and **Hong Kong**. In general, as we can observe, these high values are mostly related to **small countries and islands**, with the area being much below average(553,000km2), but also the population below average(30M people). To narrow the subset of all countries with the `population density` above average and to find bigger territories among them, we'll select only those of them, for which both of the following statements are true:

- The popualtion is above average
- the area_land is below average

In [18]:
%%sql
SELECT name, population, area_land, ROUND(population/area_land) AS population_density
  FROM facts
WHERE population NOT NULL
 AND  name <> 'Antarctica'
    AND area_land NOT NULL
    AND area_land <> 0
    AND population > (SELECT AVG(population) FROM facts
                     WHERE population NOT NULL
                    AND  name <> 'Antarctica'
                    AND area_land NOT NULL
                    AND area_land <> 0)
    AND area_land < (SELECT AVG(area_land) FROM facts
                     WHERE population NOT NULL
                    AND  name <> 'Antarctica'
                    AND area_land NOT NULL
                    AND area_land <> 0)
ORDER BY population_density DESC


 * sqlite:///factbook.db
Done.


name,population,area_land,population_density
Bangladesh,168957745,130170,1297.0
"Korea, South",49115196,96920,506.0
Japan,126919659,364485,348.0
Philippines,100998376,298170,338.0
Vietnam,94348835,310070,304.0
United Kingdom,64088222,241930,264.0
Germany,80854408,348672,231.0
Nepal,31551305,143351,220.0
Italy,61855120,294140,210.0
Uganda,37101745,197100,188.0


Our initial subset is now significantly reduced. We can conclude that among the countries with both the `population above average` and the `area below average`, the `highest population density` is related to **Bangladesh**, followed with a huge gap by **South Korea**.

Next, we are going to do the same steps for the countries with the population density below average, starting from the *BOTTOM 20*.

In [19]:
%%sql
SELECT name, population, area_land, ROUND(CAST(population AS Float)/CAST(area_land AS Float), 2) AS population_density  
FROM facts
 WHERE population NOT NULL
   AND area_land NOT NULL
    AND area_land <> 0
     AND name <> 'Antarctica'
        AND population_density < (
            SELECT AVG(population)/AVG(area_land)
              FROM facts
               WHERE population NOT NULL
                AND area_land NOT NULL
                AND area_land <> 0
                AND name <> 'Antarctica')
 ORDER BY population_density
 LIMIT 20;

 * sqlite:///factbook.db
Done.


name,population,area_land,population_density
Greenland,57733,2166086,0.03
Svalbard,1872,62045,0.03
Falkland Islands (Islas Malvinas),3361,12173,0.28
Pitcairn Islands,48,47,1.02
Mongolia,2992908,1553556,1.93
Western Sahara,570866,266000,2.15
Namibia,2212307,823290,2.69
Australia,22751014,7682300,2.96
Iceland,331918,100250,3.31
Mauritania,3596702,1030700,3.49


The absolute leaders for their `low population density` are **Greenland** and **Svalbard**.

Let's put furhter restrictions to this subset and select only those countries, for which both of the following statements are true:

- The population is below average
- The area_land is above average

In [20]:
%%sql
SELECT name, population, area_land, ROUND(population/area_land) AS population_density
  FROM facts
WHERE population NOT NULL
 AND  name <> 'Antarctica'
    AND area_land NOT NULL
    AND area_land <> 0
    AND population < (SELECT AVG(population) FROM facts
                     WHERE population NOT NULL
                    AND  name <> 'Antarctica'
                    AND area_land NOT NULL
                    AND area_land <> 0)
    AND area_land > (SELECT AVG(area_land) FROM facts
                     WHERE population NOT NULL
                    AND  name <> 'Antarctica'
                    AND area_land NOT NULL
                    AND area_land <> 0)
ORDER BY population_density


 * sqlite:///factbook.db
Done.


name,population,area_land,population_density
Greenland,57733,2166086,0.0
Mongolia,2992908,1553556,1.0
Australia,22751014,7682300,2.0
Namibia,2212307,823290,2.0
Botswana,2182719,566730,3.0
Libya,6411776,1759540,3.0
Mauritania,3596702,1030700,3.0
Kazakhstan,18157122,2699700,6.0
Central African Republic,5391539,622984,8.0
Bolivia,10800882,1083301,9.0


Hence, our second subset is also significantly reduced: now we see `big countries` with relatively `low values of population`, with **Greenland** on the top followed by **Mongolia**, **Namibia** and **Australia**.

## Further Questions
## 1. Which country has the highest growth rate?

In [21]:
%%sql
SELECT *
FROM facts
ORDER BY population_growth DESC
LIMIT 1;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


## 2. Which countries have the highest ratios of water to land? 

In [22]:
%%sql
SELECT *, area_water/area_land AS WL
FROM facts
WHERE area_water <> 0
AND WL <> 0
ORDER BY WL DESC


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,WL
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4


## 3.Which countries will add the most people to their populations next year?

In [23]:
%%sql
SELECT name, round(birth_rate - death_rate, 2) AS birth_ratio
  FROM facts
    WHERE name <> 'World'
ORDER BY birth_ratio DESC

LIMIT 3;

 * sqlite:///factbook.db
Done.


name,birth_ratio
Malawi,33.15
Uganda,33.1
Niger,33.03


## 4.Which countries have a higher death rate than birth rate?

In [24]:
%%sql
SELECT name, round(death_rate - birth_rate, 2) AS death_ratio
 FROM facts
WHERE name <> 'World'
ORDER BY death_ratio DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,death_ratio
Bulgaria,5.52
Serbia,4.58
Latvia,4.31
Lithuania,4.17
Ukraine,3.74
